# Volume Spike Agent

The ``VolumeSpikeNotifierAgent` monitors trading volume in a Uniswap V2 liquidity pool and sends notifications when volume exceeds a specified threshold, ideal for detecting whale activity or market trends. It integrates DeFiPy’s LPQuote for exits and Web3Scout’s event feeds for real-time Volume updates, supporting off-chain testing and on-chain execution.

### 📘 Notable Classes

---

* **Class**: 📘 ``defipy.agents.config.VolumeSpikeConfig`` 
    * **Purpose**: Configuration for volume threshold agent.
        * **Parameters**:
            * `volume_threshold`: Volume threshold to check (`float`).
            * `pool_address`: UniV2 pool address (`str`).
            * `provider_url`: Provider URL (eg, infura.io)(`str`).
            * `platform`: Platform where pool resides (eg, uniswap) (`str`).
            * `abi_name`: ABI handle pointing to pool contract in JSON (eg, UniswapV2Pair) (`str`).

---
         
* **Class**: 📘 ``defipy.agents.VolumeSpikeNotifierAgent`` 
    * **Purpose**: Reactive DeFi agent for determining price threshold.
        * **Parameters**:
            * `config`: Agent configuration parameters (`PriceThresholdConfig`).
    * **Methods**:
        * ``apply()``
            * **Purpose**: Apply price threshold agent
        * ``run_batch(lp: UniswapExchange, tkn: ERC20, user_nm: str, events: dict)``
            * **Purpose**: Run AI price agent on batch data 
            * **Parameters**:
                * `lp`: Swap token (`UniswapExchange`).  
                * `tkn`: Swap token (`ERC20`).
                * `user_nm`: Account name (`str`).
                * `events`: Dictionary of sync events (`dict`).
        * ``apply(lp: UniswapExchange, tkn: ERC20, user_nm: str, block_num: int)``
            * **Purpose**: Apply TVL check
            * **Parameters**:
                * `lp`: Exchange (`UniswapExchange`).  
                * `tkn`: Swap token (`ERC20`).
                * `user_nm`: Account name (`str`).
                * `block_num`: Block number (`int`).             
        * ``check_condition(lp: UniswapExchange, tkn: ERC20, threshold: float, block_num: int)``
            * **Purpose**: Check if TVL is below threshold  
            * **Parameters**:
                * `lp`: Exchange (`UniswapExchange`).  
                * `tkn`: Swap token (`ERC20`).
                * `threshold`: override config price threshold (optional) (`float`).
                * `block_num`: Block number (`int`).
        * ``get_pool_volume(lp: UniswapExchange, tkn: ERC20, block_num: int)``
            * **Purpose**: Check if TVL is below threshold  
            * **Parameters**:
                * `lp`: Exchange (`UniswapExchange`).  
                * `tkn`: Swap token (`ERC20`).
                * `block_num`: Block number (`int`).
        * ``take_mock_position(lp: UniswapExchange, tkn: ERC20, user_nm: str, amt: float)``
            * **Purpose**: Check if TVL is below threshold  
            * **Parameters**:
                * `lp`: Exchange (`UniswapExchange`).  
                * `tkn`: Swap token (`ERC20`).
                * `user_nm`: Account name (`str`).
                * `amt`: Mock token amount (`float`).
        * ``withdraw_mock_position(lp: UniswapExchange, tkn: ERC20, user_nm: str, lp_amt: float)``
            * **Purpose**: Check if TVL is below threshold  
            * **Parameters**:
                * `lp`: Exchange (`UniswapExchange`).  
                * `tkn`: Swap token (`ERC20`).
                * `user_nm`: Account name (`str`).
                * `lp_amt`: Mock LP amount (`float`).
        * ``update_mock_pool(lp: UniswapExchange, cur_block: int)``
            * **Purpose**: Check if TVL is below threshold  
            * **Parameters**:
                * `lp`: Exchange (`UniswapExchange`).  
                * `cur_block`: Current block number (`int`).
        * ``prime_mock_pool(start_block: int, user_nm: str)``
            * **Purpose**: Initialize off-chain pool using information in start_block
            * **Parameters**:
                * `start_block`: Start block (`int`).
                * `user_nm`: Account name of mock off-chain pool (`str`). 
        * ``get_w3()``
            * **Purpose**: Getter function for web3.py connector object
        * ``get_abi()``
            * **Purpose**: Getter function for ABI json data
        * ``get_contract_instance()``
            * **Purpose**: Getter function for pool contract instance
        * ``get_lp_data()``
            * **Purpose**: Getter function for pool data associated with `pool_address` from config


In [1]:
from defipy import *
from web3scout import *

In [3]:
volume_threshold = 1000
pair_address = "0xB4e16d0168e52d35CaCD2c6185b44281Ec28C9Dc"
provider_url = "https://mainnet.infura.io/v3/9624e3e5c40f4ac3958b79fa5aa2562d"
platform = Platform.AGNOSTIC
abi_name = JSONContract.UniswapV2Pair
user_position = 10

config = VolumeSpikeConfig(
    volume_threshold = volume_threshold,
    pool_address = pair_address,
    provider_url = provider_url,
    platform = platform,
    abi_name = abi_name,
    user_position = user_position
)

agent = VolumeSpikeNotifierAgent(config)
agent.init()

print(f"Monitoring TVL changes @ pool address {pair_address}")

Monitoring TVL changes @ pool address 0xB4e16d0168e52d35CaCD2c6185b44281Ec28C9Dc


In [4]:
abi = ABILoad(platform, abi_name)
connect = ConnectW3(provider_url)
connect.apply()

last_block = connect.get_w3().eth.block_number
start_block = last_block - 100

# Grab batch sync events from pool
rEvents = RetrieveEvents(connect, abi)
events = rEvents.apply(EventType.SWAP, address = pair_address, start_block=start_block, end_block=last_block)
df_events = rEvents.to_dataframe(events)
df_events.head(2)

,blockNumber,event,address,blockHash,logIndex,transactionHash,transactionIndex,args
0,23192028,Swap,0xB4e16d0168e52d35CaCD2c6185b44281Ec28C9Dc,0xc187557bd7cbc69d489c61b7b356ff7bc47d79ca9fe2...,311,0x82283ff0fa4616f5f0aac75fd6da4647b2b79294e658...,70,{'sender': '0x66a9893cC07D91D95644AEDD05D03f95...
1,23192031,Swap,0xB4e16d0168e52d35CaCD2c6185b44281Ec28C9Dc,0x4eb0287fa39478fac4992f1989967575a844a1e62528...,111,0x0a3cbd5d9d11a2466f2f748bee75565466947a21714c...,20,{'sender': '0x51C72848c68a965f66FA7a88855F9f77...


### Agent Run #1

In [5]:
tkn0 = agent.get_lp_data().tkn0
tkn1 = agent.get_lp_data().tkn1
lp = agent.prime_mock_pool(start_block, 'user')
lp.summary()

Exchange USDC-WETH (LP)
Reserves: USDC = 14753285.214759, WETH = 3490.4443349549842
Liquidity: 0.0844553765492715 



In [6]:
agent.run_batch(lp, tkn0, 'user', events)

Block 23192028: Volume threshold condition met for USDC-WETH LP
Block 23192031: Volume (USDC) = 12194.247065504365, outside threshold 1000.0
Block 23192037: Volume (USDC) = 1048.4436339924819, outside threshold 1000.0
Block 23192053: Volume threshold condition met for USDC-WETH LP
Block 23192054: Volume threshold condition met for USDC-WETH LP
Block 23192057: Volume threshold condition met for USDC-WETH LP
Block 23192059: Volume threshold condition met for USDC-WETH LP
Block 23192064: Volume (USDC) = 6713.7046538946815, outside threshold 1000.0
Block 23192065: Volume (USDC) = 26576.830316453408, outside threshold 1000.0
Block 23192065: Volume threshold condition met for USDC-WETH LP
Block 23192065: Volume threshold condition met for USDC-WETH LP
Block 23192066: Volume (USDC) = 3841.7835243902446, outside threshold 1000.0
Block 23192073: Volume (USDC) = 2795.9323429773003, outside threshold 1000.0
Block 23192081: Volume threshold condition met for USDC-WETH LP
Block 23192082: Volume thr